# Natureness Image Recognition

Train and evaluation of multiple image recognition models on 2 datasets: city, nature.

## Goals

- Train models on 2 datasets: city, nature
- Evaluate models on 2 datasets: city, nature
- Compare the performance of the models on the 2 datasets, even with their respective evaluation datasets
- Train more image recognition models with transfer learning